In [1]:
import numpy as np
import scipy as sp
from scipy import linalg as LA # differ from cytnx.linalg
import sys
sys.path.append("/usr/local/")
import cytnx
from cytnx import cytnx_extension as cyx

In [4]:
##### Lets initialize some tensors in Python/Numpy
import numpy as np
### tensor with randomly generated entries, order 3, dims: 2-by-3-by-4
A = np.arange(24).reshape(2,3,4)
Act = cytnx.arange(24).reshape(2,3,4)
print(LA.norm(A-Act.numpy()))


### identity matrix, order 2, dims: 5-by-5 
B = np.eye(5,5)
Bct = cytnx.zeros([5,5])
for i in range(5):
    Bct[i,i] = 1
    
print(LA.norm(B- Bct.numpy()))

### tensor of 1's, order 4, dims: 2-by-4-by-2-by-4
C = np.ones((2,4,2,4))
Cct = cytnx.ones([2,4,2,4])
print(LA.norm(C-Cct.numpy()))

### matrix of 0's, order 2, dims: 3-by-5
D = np.zeros((3,5))
Dct = cytnx.zeros([3,5])
print(LA.norm(D-Dct.numpy()))




0.0
0.0
0.0
0.0


In [15]:
##### Ex.1.2(a):Transpose
A = np.arange(2**3).reshape(2,2,2)
Atilda = A.transpose(1,2,0)

Act = cytnx.arange(2**3).reshape(2,2,2)
Act = Act.permute(1,2,0)

print(LA.norm(Atilda-Act.numpy())) # bad


0.0


In [17]:
##### Ex.1.2(b):Reshape
B = np.arange(64).reshape(4,4,4)
Btilda = B.reshape(4,4**2)
Bct = cytnx.arange(64). reshape(4,4,4)
BtildaCt = Bct.reshape(4,4**2)
print(LA.norm(Btilda-BtildaCt.numpy()))

0.0


In [18]:
##### Ex.1.3(a): Binary Tensor Contraction
d = 2
A = np.arange(16).reshape(d,d,d,d)  
B = np.arange(16).reshape(d,d,d,d)  

Ap  = A.transpose(0,2,1,3);  Bp = B.transpose(0,3,1,2)
App = Ap.reshape(d**2,d**2); Bpp = Bp.reshape(d**2,d**2)
Cpp = App @ Bpp;             C   = Cpp.reshape(d,d,d,d)

##### Using Cytnx
Act = cytnx.arange(16).reshape(d,d,d,d)  
Bct = cytnx.arange(16).reshape(d,d,d,d)
ApCt  = Act.permute(0,2,1,3);  BpCt = Bct.permute(0,3,1,2)
AppCt = ApCt.reshape(d**2,d**2); BppCt = BpCt.reshape(d**2,d**2)
CppCt = cytnx.linalg.Matmul(AppCt, BppCt);             Cct   = CppCt.reshape(d,d,d,d)


print(LA.norm(C - Cct.numpy()))

0.0


In [24]:
##### Ex.1.5(b): Contraction using ncon
from ncon import ncon # Need ncon.py in the directory

d = 3
A = np.arange(27).reshape(d,d,d); B = np.arange(81).reshape(d,d,d,d)
C = np.arange(27).reshape(d,d,d); D = np.arange(9).reshape(d,d)

TensorArray = [A,B,C,D]

IndexArray = [[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]

E = ncon(TensorArray,IndexArray)

######### Using Cytnx

Act = cytnx.arange(27).reshape(d,d,d); Bct = cytnx.arange(81).reshape(d,d,d,d)
Cct = cytnx.arange(27).reshape(d,d,d); Dct = cytnx.arange(9).reshape(d,d)
Act = cyx.CyTensor(Act,2)
Act.set_labels([1,-2,2])
Bct = cyx.CyTensor(Bct,2)
Bct.set_labels([-1,1,3,4])
Cct = cyx.CyTensor(Cct,2)
Cct.set_labels([5,3,2])
Dct = cyx.CyTensor(Dct,2)
Dct.set_labels([4,5])

# Act.print_diagram()
# Bct.print_diagram()
# Cct.print_diagram()
# Dct.print_diagram()
# Result = cyx.Contract(A)

Ect = cyx.Contract(cyx.Contract(Act,Bct),cyx.Contract(Cct,Dct))
Ect.print_diagram()

print(type(Ect))
Ect = Ect.get_block()
print(type(Ect))
Ect = Ect.permute(1,0)
print(LA.norm(E - Ect.numpy()))


-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -2 ____| 3         3 |____ -1 
           \             /     
            -------------      
<class 'cytnx.cytnx.cytnx_extension_c.CyTensor'>
<class 'cytnx.cytnx.Tensor'>
0.0
